# advent of code 2024 - [day 19](https://adventofcode.com/2024/day/19)

### Parsing

In [66]:
import os

NEO4J_URI = os.environ['NEO4J_URI']
NEO4J_USERNAME = os.environ['NEO4J_USERNAME']
NEO4J_PASSWORD = os.environ['NEO4J_PASSWORD']

In [67]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

from neo4j import GraphDatabase
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

import pandas as pd

2.13.0


In [68]:
def parse_towels(file='input.txt'):
    file = open(file, 'r')
    for line in file:
        for ix, x in enumerate(line.strip().split(',')):
            yield {'towel_id': ix, 'towel': list(x.strip())}
        break

def parse_patterns(file='input.txt'):
    file = open(file, 'r')
    for ix, line in enumerate(file):
        if ix < 2:
            continue
        yield {'pattern_id': ix - 2, 'pattern':tuple(line.strip())}


def parse_patterns_fine(file='input.txt'):
    for t in parse_patterns(file):
        for sx, stripe in enumerate(t['pattern']):
            yield {'pattern_id': t['pattern_id'], 'stripe_num': sx, 'color': stripe} 

In [69]:
filename = 'input.txt'
#filename = 'test.txt'


patterns_elements = list(parse_patterns_fine(filename))
towels = list(parse_towels(filename))


## Part 1 with Neo4j

In [70]:
clean_queries = [
"CALL apoc.schema.assert({},{});",
"""MATCH (n)
CALL {WITH n DETACH DELETE n}
IN TRANSACTIONS OF 10000 ROWS;""",
"""CALL gds.graph.list()
YIELD graphName
WITH graphName AS g
CALL (g) {CALL gds.graph.drop(g) YIELD graphName RETURN graphName}
WITH graphName RETURN graphName;"""]

for q in clean_queries:
    gds.run_cypher(q, {})

In [71]:
gds.run_cypher('CREATE INDEX stripe_pattern_id_stripe_num_color IF NOT EXISTS FOR (s:Stripe) ON (s.pattern_id, s.stripe_num, s.color)')

""


In [72]:
build_q= ["""
UNWIND $patterns_elements AS stripe
CREATE (s:Stripe)
SET s += stripe
""",
"""
UNWIND $towels AS towel
CREATE (t:Towel)
SET t += towel

"""
,
"""
MATCH (s:Stripe)
WITH s.pattern_id AS pattern, collect(s) AS stripes
ORDER BY pattern
CALL apoc.nodes.link(stripes,'NEXT')
""",
"""MATCH (s:Stripe) WHERE NOT EXISTS {()-[:NEXT]->(s)}
SET s:First""",
"""MATCH (s:Stripe) WHERE NOT EXISTS {(s)-[:NEXT]->()}
SET s:Last""",
"""
MATCH (t:Towel)
WITH t,
  apoc.text.join([x IN t.towel  | "(:Stripe {color:'" + x + "'})"], '-[:NEXT]->') AS gpm_pattern
WITH
  t,
  "MATCH (t:Towel {towel_id: "+ t.towel_id +"})
  MATCH path = " + gpm_pattern + "
  WITH t, nodes(path)[0] AS match_first, nodes(path)[-1] AS match_last
  CREATE (m:Match)
  MERGE (t)-[:MATCHES]->(m)
  MERGE (m)-[:FROM]->(match_first)
  MERGE (m)-[:TO]->(match_last)" AS query
WITH t, query
SET t.match_query = query
"""
]
for q in build_q:
    gds.run_cypher(q, {'patterns_elements':patterns_elements, 'towels': towels})

In [73]:
match_queries = gds.run_cypher("MATCH (t:Towel) RETURN collect(t.match_query) AS match_queries")['match_queries'][0]
for q in match_queries:
    gds.run_cypher(q, {})

In [74]:
gds.run_cypher("""
MATCH (first:First)
CALL (first) {
MATCH path = (first)(()<-[:FROM]-()-[:TO]->()-[:NEXT]->())+()<-[:FROM]-()-[:TO]->(last:Last)
RETURN path LIMIT 1
} IN CONCURRENT TRANSACTIONS OF 100 ROWS
RETURN count(DISTINCT first) AS part1""")

,part1
0,340


## Part 2 with dynamic programming



In [75]:
def parse_towels(file='input.txt'):
    file = open(file, 'r')
    for line in file:
        for ix, x in enumerate(line.strip().split(',')):
            yield x.strip()
        break

def parse_patterns(file='input.txt'):
    file = open(file, 'r')
    for ix, line in enumerate(file):
        if ix < 2:
            continue
        yield line.strip()

In [76]:
from functools import cache

@cache
def num_solutions(pattern, towels):
    acc = 0
    for towel in towels:
        if pattern == towel:
            acc += 1
        elif pattern.startswith(towel):
            l = len(towel)
            acc += num_solutions(pattern[l:], towels)
        else:
            pass
    return acc

part2 = 0
towels = tuple(parse_towels(filename))

for pattern in parse_patterns(filename):
    part2 += num_solutions(pattern, towels)

part2


717561822679428